# Descriptor

* What is descriptor
* Basic example
* Practice
* When to use

## What is descriptor

`Descriptor protocol`: if a class implements at least of the magic methods: `__get__`, `__set__` and `__delete__`, it is a descriptor.

In [ ]:
class Descriptor:
    def __get__(self, obj, objtype=None) -> "value":
        ...
    
    def __set__(self, obj, value) -> None:
        ...
    
    def __delete__(self, obj) -> None:
        ...

## Basic Example

The class `Attribute` is a simple descriptor.

In [ ]:
class Attribute:
    def __set_name__(self, owner, name):  
        """create an attribute name for the taget varible"""
        print(f"setting name '{name}' on '{owner}'")
        self.private_name = f"_{name}"

    def __get__(self, obj, objtype=None):
        print(f"getting on '{obj}' of '{objtype}' from Attribute.__get__")
        return getattr(obj, self.private_name)
    
    def __set__(self, obj, value):
        print(f"setting '{value}' on '{obj}' from Attribute.__set__")
        setattr(obj, self.private_name, value)

    def __delete__(self, obj):
        print(f"deleting on '{obj}' from Attribute.__delete__")
        delattr(obj, self.private_name)

To use this descriptor, we need to instantiate it as a class variable.

In [ ]:
class Sample:
    name = Attribute()  # this will call Attribute.__set_name__

    def __init__(self, name):
        self.name = name

Then we can invoke the descriptor by get/set/delete the attribute `name`. 

In [ ]:
# set `name` via descriptor, calling Attribute.__set__
s = Sample("test")
s.__dict__

In [ ]:
# get `name` via descriptor, calling Attribute.__get__
s.name

In [ ]:
# delete `name` via descriptor, calling Attribute.__delete__
del s.name
s.__dict__

## Practices

### Type validator
One of the usage of descriptor is type validator. By nature, Python is a dynamically-typed language. It will not check the type until you use it.

#### Problem
We have a class user which accept user's name and his age which should be `str` and `int`.

In [ ]:
class User:
    first_name: str = None
    last_name: str = None
    age: int = None

    def __init__(self, first_name: str, last_name: str, age: int):
        self.first_name = first_name
        self.last_name = last_name
        self.age = age

    def info(self):
        return "full name:" + self.first_name + " " + self.last_name

    def age_of_next_year(self):
        return self.age + 1

It works fine, if you give the right type.

In [ ]:
alice = User(first_name="Alice", last_name="A", age=20)
print(alice.info())
print(alice.age_of_next_year())

But if you give a wrong type, the constructor will not tell you that you are wrong.

In [ ]:
bob = User(first_name="Bob", last_name=0, age="30")

You will only be telled the error when you call the relevant methods.

In [ ]:
bob.info()

In [ ]:
bob.age_of_next_year()

#### Solution
To overcome this, of course, we can add some type checking code in the constructor. But it is not convient.

In [ ]:
class User:
    first_name: str = None
    last_name: str = None
    age: int = None

    def __init__(self, first_name: str, last_name: str, age: int):
        if not isinstance(first_name, str):
            raise TypeError(f"first_name has wrong type, expected '{type(first_name)}'")
        if not isinstance(last_name, str):
            raise TypeError(f"last_name has wrong type, expected '{type(last_name)}'")
        if not isinstance(age, int):
            raise TypeError(f"age has wrong type, expected '{type(age)}'")
        self.first_name = first_name
        self.last_name = last_name
        self.age = age

    def info(self):
        return "full name:" + self.first_name + " " + self.last_name

    def age_of_next_year(self):
        return self.age + 1

So descriptor can help here. We create some validator classes.

In [ ]:
class TypeValidator:
    def __set_name__(self, obj, name):
        self.name = name
        self.private_name = f"_{name}"
    
    def __get__(self, obj, objtype=None):
        return getattr(obj, self.private_name)

    def __set__(self, obj, value):
        self.validate(value)
        setattr(obj, self.private_name, value)

    def validate(self, value):
        raise NotImplementedError("Not implemented in TypeValidator")

class String(TypeValidator):
    def validate(self, value):
        if not isinstance(value, str):
            raise TypeError(f"{self.name} has wrong type, expected '{type(value)}'")

class Int(TypeValidator):
    def validate(self, value):
        if not isinstance(value, int):
            raise TypeError(f"{self.name} has wrong type, expected '{type(value)}'")

Apply them to the `User` class.

In [ ]:
class User:
    first_name: str = String()
    last_name: str = String()
    age: int = Int()

    def __init__(self, first_name: str, last_name: str, age: int):
        self.first_name = first_name
        self.last_name = last_name
        self.age = age

    def info(self):
        return "full name:" + self.first_name + " " + self.last_name

    def age_of_next_year(self):
        return self.age + 1

Let's do some tests.

In [ ]:
# it should still work
alice = User(first_name="Alice", last_name="A", age=20)
print(alice.info())
print(alice.age_of_next_year())

In [ ]:
# this should fail from constructor
bob = User(first_name="Bob", last_name=0, age="30")

## Close thought

### Descriptor types

There are 2 types of descripors: `data descriptor` and `non-data descriptor`. There is difference on the attribute look up process.

cf. [descriptor how to](https://docs.python.org/3/howto/descriptor.html#descriptor-protocol)

### Descriptor usage in python

Descriptor is also used in Python it-self, like methods(including @staticmethod, @classmethod), property...
It is also used in mecanisme like ORM...

### When we need it

In most cases, it is the owner class's job to figure out how to find the attributes.

But descriptor makes it different. The descripter takes over this job, it will find the attribute.

It can be useful when we want to add some additional treatment before saving an attribute, like type validation, modification of the saving data... Often, these treatments are expensive/inconvient to generalize if we implement the in the owner class. (I used it in [http_testing framework](https://github.com/heqile/http_testing/blob/main/http_testing/assertion_elements/assertion_attribute_base.py))

But for daily developpement, we may not often use it. But it is good to know when you need.